# HW6 Notebook
Name: **Tony Nguyen**

Class: CPSC 322, 

Term: Fall 2023

# 1. Load libraries and datasets

 Import the data table and utility functions.

In [2]:
from data_table import *
from data_learn import *
from data_eval import *
from data_util import *

# 2. Auto MPG Data Analysis

Load and clean auto data.

In [3]:
auto = DataTable(['mpg','cyls','disp','hp','weight','accl','year','origin','name'])
auto.load('auto-mpg.txt')
auto.rows(range(10))

  mpg    cyls    disp    hp    weight    accl    year    origin  name
-----  ------  ------  ----  --------  ------  ------  --------  -------------------------
   18       8     307   130      3504    12        70         1  chevrolet chevelle malibu
   15       8     350   165      3693    11.5      70         1  buick skylark 320
   18       8     318   150      3436    11        70         1  plymouth satellite
   16       8     304   150      3433    12        70         1  amc rebel sst
   17       8     302   140      3449    10.5      70         1  ford torino
   15       8     429   198      4341    10        70         1  ford galaxie 500
   14       8     454   220      4354     9        70         1  chevrolet impala
   14       8     440   215      4312     8.5      70         1  plymouth fury iii
   14       8     455   225      4425    10        70         1  pontiac catalina
   15       8     390   190      3850     8.5      70         1  amc ambassador dpl

In [4]:
print("Number of rows before cleaning:", auto.row_count())

Number of rows before cleaning: 317


In [5]:
auto = remove_duplicates(auto)
print("Number of rows after removing duplicated rows:", auto.row_count())

Number of rows after removing duplicated rows: 316


In [6]:
auto = remove_missing(auto, auto.columns())
print("Number of rows after removing missing values:", auto.row_count())

Number of rows after removing missing values: 307


## Step 1: k-NN versus Naive Bayes 

### 1.1 Discretize the mpg value in the auto table using three equal-width bins

In [7]:
auto_copy = auto.copy()
mpg_value = auto_copy.get_column_data('mpg')
mpg_value = sorted(mpg_value)
bin_width = (mpg_value[-1] - mpg_value[0]) / 3
cut_points = [mpg_value[0] + bin_width, mpg_value[0] + 2 * bin_width]
discretize(auto_copy, 'mpg', cut_points)

In [8]:
auto_copy.rows(range(20))

  mpg    cyls    disp    hp    weight    accl    year    origin  name
-----  ------  ------  ----  --------  ------  ------  --------  ----------------------------
    1       8     307   130      3504    12        70         1  chevrolet chevelle malibu
    1       8     350   165      3693    11.5      70         1  buick skylark 320
    1       8     318   150      3436    11        70         1  plymouth satellite
    1       8     304   150      3433    12        70         1  amc rebel sst
    1       8     302   140      3449    10.5      70         1  ford torino
    1       8     429   198      4341    10        70         1  ford galaxie 500
    1       8     454   220      4354     9        70         1  chevrolet impala
    1       8     440   215      4312     8.5      70         1  plymouth fury iii
    1       8     455   225      4425    10        70         1  pontiac catalina
    1       8     390   190      3850     8.5      70         1  amc ambassador dpl
    1    

### 1.2 Normalize the weight (weight) and displacement (disp) attributes

In [9]:
normalize(auto_copy, 'weight')
normalize(auto_copy, 'disp')

### 1.3 Evaluate knn using stratified k-fold cross validation (i.e., your knn_stratified() function) to predict mpg labels using 10 folds, a knn k-value of 7, majority voting, and only the weight and displacement attributes (as numeric columns). Display the resulting confusion matrix.

In [10]:
result = knn_stratified(auto_copy, 10, 'mpg', majority_vote, 7, ['weight', 'disp'])
print(result)

actual      1    2    3    total
--------  ---  ---  ---  -------
1         162    0    0      162
2         123    0    0      123
3          22    0    0       22
total     307    0    0      307


### 1.4 Compute accuracy, precision, recall, and the f-measure over the resulting confusion matrix and display each.

Number of non-empty classes

In [11]:
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

Non-empty labels count: 3


Find accuracy

In [12]:
total_accuracy = 0
total_accuracy_after_iteration = 0
label_list = result.get_column_data('actual')
if label_list[-1] == 'total':
    label_list = label_list[:-1]
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
total_accuracy_after_iteration += acc
print("Average accuracy:", acc)

Average accuracy: 0.6851248642779587


Find precision

In [13]:
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

p_prediced is 0 for label 2 . Subtracted 1 from non_empty_label.
p_prediced is 0 for label 3 . Subtracted 1 from non_empty_label.
Precision: 0.1758957654723127


Find recall

In [14]:
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

Recall: 1.0


Find f-score

In [15]:
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

Macro-averaging F-score: 0.29916897506925205


### 1.5 Repeat step 3 using the same parameters but using naive-bayes instead of knn (i.e., your naive_bayes_stratified() function). Be sure to use weight and displacement as continuous attributes. Display the resulting confusion matrix.

In [16]:
result = naive_bayes_stratified(auto_copy, 10, 'mpg', ['weight', 'disp'])

In [17]:
print(result)

actual      1    2    3    total
--------  ---  ---  ---  -------
1         143   18    1      162
2          14   77   32      123
3           0    5   17       22
total     157  100   50      307


### 1.6 Compute accraccy, precision, recall, and the f-measure over the resulting confusion matrix and display each.

Number of non-empty classes

In [18]:
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

Non-empty labels count: 3


Find accuracy

In [19]:
total_accuracy = 0
total_accuracy_after_iteration = 0
label_list = result.get_column_data('actual')
if label_list[-1] == 'total':
    label_list = label_list[:-1]
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
total_accuracy_after_iteration += acc
print("Average accuracy:", acc)

Average accuracy: 0.8479913137893593


Find precision

In [20]:
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

Precision: 0.6736093418259023


Find recall

In [21]:
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

Recall: 0.7604865274241966


Find f-measure

In [22]:
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

Macro-averaging F-score: 0.7144164350372894


### 1.7 Comparing knn and naive-bayes stratified

From the above results, the differences between the two is quite significant.
* The accuracy of knn is 0.68 while the accuracy of naive-bayes is 0.85.
* The precision of knn is 0.18 while the precision of naive-bayes is 0.67.
* The recall of knn is 1 while the recall of naive-bayes is 0.76.
* The f-measure of knn is 0.3 while the f-measure of naive-bayes is 0.71.

I find the main reason that makes such a different result is because of the k-stratified cross validation selection method.
* For some reason, while it was picking samples for the knn test, it was picking samples that were not representative of the whole dataset.
* Although the test set preserved the original class distribution, it might not be enough to produce a good test result as it was not representative of the whole dataset.
* Thus, the model for knn was underfitting the data and produced a bad result.
* Naive-bayes produced a much better result.

## Step 2: Experimentation with Auto MPG Data Classification

### 2.1 Try to factor in more attributes to see if the results improve.

kNN result
* Pick k_fold=10, k=20 and bin=5

In [23]:
# Discretize
auto_copy = auto.copy()
mpg_value = auto_copy.get_column_data('mpg')
mpg_value = sorted(mpg_value)
bin_width = (mpg_value[-1] - mpg_value[0]) / 3
cut_points = [mpg_value[0] + bin_width, mpg_value[0] + 2 * bin_width]
discretize(auto_copy, 'mpg', cut_points)

# Normalize
normalize(auto_copy, 'cyls')
normalize(auto_copy, 'disp')
normalize(auto_copy, 'hp')
normalize(auto_copy, 'weight')
normalize(auto_copy, 'accl')


# kNN
result = knn_stratified(auto_copy, 10, 'mpg', majority_vote, 20, ['cyls', 'disp', 'hp', 'weight', 'accl'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      1    2    3    total
--------  ---  ---  ---  -------
1         149   13    0      162
2          23   97    3      123
3           0   20    2       22
total     172  130    5      307

Non-empty labels count: 3
Average accuracy: 0.8718783930510314
Precision: 0.6708109719737627
Recall: 0.5997600211692352
Macro-averaging F-score: 0.6332988946273842


Try it again with Naive Bayes
* k=10

In [35]:
# Discretize
auto_copy = auto.copy()
mpg_value = auto_copy.get_column_data('mpg')
mpg_value = sorted(mpg_value)
bin_width = (mpg_value[-1] - mpg_value[0]) / 3
cut_points = [mpg_value[0] + bin_width, mpg_value[0] + 2 * bin_width]
discretize(auto_copy, 'mpg', cut_points)

# Normalize
normalize(auto_copy, 'cyls')
normalize(auto_copy, 'disp')
normalize(auto_copy, 'hp')
normalize(auto_copy, 'weight')
normalize(auto_copy, 'accl')


# kNN
result = naive_bayes_stratified(auto_copy, 10, 'mpg', ['cyls', 'disp', 'hp', 'weight', 'accl'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      1    2    3    total
--------  ---  ---  ---  -------
1         145    9    8      162
2          12   12   99      123
3           0    1   21       22
total     157   22  128      307

Non-empty labels count: 3
Average accuracy: 0.7198697068403909
Precision: 0.5443613081451457
Recall: 0.6490560528500908
Macro-averaging F-score: 0.5921164104640683


### 2.2 Try it again but with weighting in origin and year as well

kNN result

In [25]:
# Discretize
auto_copy = auto.copy()
mpg_value = auto_copy.get_column_data('mpg')
mpg_value = sorted(mpg_value)
bin_width = (mpg_value[-1] - mpg_value[0]) / 5
cut_points = [mpg_value[0] + bin_width, mpg_value[0] + 2 * bin_width, mpg_value[0] + 3 * bin_width, mpg_value[0] + 4 * bin_width]
discretize(auto_copy, 'mpg', cut_points)

# Normalize
normalize(auto_copy, 'cyls')
normalize(auto_copy, 'disp')
normalize(auto_copy, 'hp')
normalize(auto_copy, 'weight')
normalize(auto_copy, 'accl')


# kNN
result = knn_stratified(auto_copy, 10, 'mpg', majority_vote, 20, ['cyls', 'disp', 'hp', 'weight', 'accl'], ['year', 'origin'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      1    2    3    4    5    total
--------  ---  ---  ---  ---  ---  -------
1          60   14    0    0    0       74
2          17   80   19    1    0      117
3           1   15   60    2    0       78
4           0    3   10   22    0       35
5           0    0    2    1    0        3
total      78  112   91   26    0      307

Non-empty labels count: 5
Average accuracy: 0.8892508143322475
p_prediced is 0 for label 5 . Subtracted 1 from non_empty_label.
Precision: 0.5978021978021978
Recall: 0.7230934230934231
Macro-averaging F-score: 0.6545056713087872


Naive Bayes result

In [26]:
# Discretize
auto_copy = auto.copy()
mpg_value = auto_copy.get_column_data('mpg')
mpg_value = sorted(mpg_value)
bin_width = (mpg_value[-1] - mpg_value[0]) / 5
cut_points = [mpg_value[0] + bin_width, mpg_value[0] + 2 * bin_width, mpg_value[0] + 3 * bin_width, mpg_value[0] + 4 * bin_width]
discretize(auto_copy, 'mpg', cut_points)

# Normalize
normalize(auto_copy, 'cyls')
normalize(auto_copy, 'disp')
normalize(auto_copy, 'hp')
normalize(auto_copy, 'weight')
normalize(auto_copy, 'accl')


# kNN
result = naive_bayes_stratified(auto_copy, 10, 'mpg', ['cyls', 'disp', 'hp', 'weight', 'accl'], ['origin', 'year'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      1    2    3    4    5    total
--------  ---  ---  ---  ---  ---  -------
1          68    6    0    0    0       74
2          19   74   16    8    0      117
3           1    6   21   50    0       78
4           0    0    1   32    2       35
5           0    0    0    3    0        3
total      88   86   38   93    2      307

Non-empty labels count: 5
Average accuracy: 0.854071661237785
Precision: 0.5059819978918175
Recall: 0.5469828069828069
Macro-averaging F-score: 0.5256841486218317


### 2.3 Observations

* After changing the bin size to 5, the result for kNN improved a lot.
* The accuracy went to 0.89, which was a significant improvement from the previous result.
* The precision went to 0.60, which was even better than Naive Bayes running at the same parameter.
* The pattern repeats itself for recall and f-score.
* So, it means that it was not the kNN algorithm that worked poorly, but the parameter size was too small for the algorithm to work properly.
* Increasing the sizes of the parameters see a better result, and even more trustworthy than Naive Bayes as its prediction, recall, and f-score were all better than Naive Bayes.

# 2. Titanic Data Analysis

Load the titanic data set below. The attributes are *class*, *age*, *gender*, and *survival*.

In [27]:
titanic = DataTable(['class', 'age', 'gender', 'survival'])
titanic.load('titanic.txt')
titanic.rows(range(10))

class    age    gender    survival
-------  -----  --------  ----------
crew     adult  female    yes
first    adult  male      no
first    adult  male      no
third    adult  female    no
second   adult  female    yes
third    adult  female    yes
second   adult  male      yes
second   adult  female    yes
crew     child  female    no
third    adult  female    no

### 2.1 Classify survival using knn

In [36]:
# kNN
titanic_copy = titanic.copy()
result = knn_stratified(titanic_copy, 10, 'survival', majority_vote, 7, [], ['class', 'age', 'gender'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      no    yes    total
--------  ----  -----  -------
no           0    711      711
yes          0   1490     1490
total        0   2201     2201

Non-empty labels count: 2
Average accuracy: 0.6769650159018628
p_prediced is 0 for label no . Subtracted 1 from non_empty_label.
Precision: 0.3384825079509314
Recall: 1.0
Macro-averaging F-score: 0.5057705363204344


### 2.2 Classify survival using naive bayes

In [37]:
# kNN
titanic_copy = titanic.copy()
result = naive_bayes_stratified(titanic_copy, 10, 'survival', [], ['class', 'age', 'gender'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      no    yes    total
--------  ----  -----  -------
no         349    362      711
yes        126   1364     1490
total      475   1726     2201

Non-empty labels count: 2
Average accuracy: 0.7782825988187188
Precision: 0.7625016771360615
Recall: 0.7031470940824437
Macro-averaging F-score: 0.7316225401880811


### 2.3 Observations

* Again, performing k-stratified cross validation on naive bayes produced a better result, as the algorithm works better in choosing the test set that works well with the respective algorithm.
* However, as seen above, it might be caused by the size parameter of the kNN algorithm. The result can definitely be inmproved if we increase the size of the parameter.
* The test set is more representative while performing naive bayes, as shown in the confusion matrix.
* In terms of accuracy, both knn and naive bayes produced a pretty decent result, with naive bayes being better at 0.78 and knn at 0.68.
* However, as the kNN algorithm is sensitive to skew. As a result, the precision and recall is much lower than naive bayes.

# 3. Student Stress Data Analysis

Load the student stress data set below. The attributes are given below in column order, where the short name to use is given in parenthesis: 
1. sleep_quality (sleep)
2. living_conditions (living)
3. basic_needs (basics)
4. academic_performance (academic)
5. study_load (study)
6. future_career_concerns (career)
7. social_support (social)
8. extracurricular_activities (extra)
9. stress_level (stress)

In [30]:
stress = DataTable(['sleep', 'living', 'basics', 'academic', 'study', 'career', 'social', 'extra', 'stress'])
stress.load('student-stress.txt')
stress.rows(range(10))

  sleep    living    basics    academic    study    career    social    extra    stress
-------  --------  --------  ----------  -------  --------  --------  -------  --------
      2         3         2           3        2         3         2        3         1
      1         1         2           1        4         5         1        5         2
      2         2         2           2        3         2         2        2         1
      1         2         2           2        4         4         1        4         2
      5         2         3           4        3         2         1        0         1
      1         2         1           2        5         5         1        4         2
      4         4         4           5        1         1         3        2         0
      1         1         1           1        3         4         1        4         2
      2         3         3           3        3         3         3        2         1
      1         5         2     

## Step 1: Initial kNN and Naive Bayes Evaluation

*TODO: Similar to the titanic analysis above, use stratified k-fold cross validation to evaluate knn and naive bayes for predicting student stress level (the stress attribute) using the other table attributes as categorical values. For both evaluations use 10 folds, and for knn use a k-value of 7 and majority voting. Give your resulting confusion matrices as well as accuracy, precision, recall, and f-measure values.*

Note that depending on your implementations, it may take some time for Python to finish running your evaluations.

### 1.1 kNN

In [31]:
# kNN
stress_copy = stress.copy()
result = knn_stratified(stress_copy, 10, 'stress', majority_vote, 7, [], ['sleep', 'living', 'basics', 'academic', 'study', 'career', 'social', 'extra'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      0    1    2    total
--------  ---  ---  ---  -------
0         373    0    0      373
1         126  231    1      358
2          69    0  300      369
total     568  231  301     1100

Non-empty labels count: 3
Average accuracy: 0.8812121212121212
Precision: 0.8844559605696193
Recall: 0.8194198422431151
Macro-averaging F-score: 0.850696702757969


### 1.2 Naive Bayes

In [32]:
# kNN
stress_copy = stress.copy()
result = naive_bayes_stratified(stress_copy, 10, 'stress', [], ['sleep', 'living', 'basics', 'academic', 'study', 'career', 'social', 'extra'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      0    1    2    total
--------  ---  ---  ---  -------
0         322   16   35      373
1          20  314   24      358
2          22   15  332      369
total     364  345  391     1100

Non-empty labels count: 3
Average accuracy: 0.9199999999999999
Precision: 0.8812883904955516
Recall: 0.8800315822789683
Macro-averaging F-score: 0.8806595379829616


### 2.3 Observations

* The results for both kNN and Naive Bayes are pretty impressive. They both yield a very high accuracy, precision, recall, and f-measure, with Naive Bayes has a bit higher statistics.

## Step 2: Experimentation with Student Stress Data

*TODO: Rerun your evaluations in Step 1 by experimenting with different knn k-values and folds (similar to the Auto MPG steps). Display, comment on, and analyze the results of your experiments.*

### 2.1 Run it again but at a lower number of attributes

* Not factoring social and extra attributes, meaning work and professional life only

kNN result
* Increase k to 10

In [33]:
# kNN
stress_copy = stress.copy()
result = knn_stratified(stress_copy, 10, 'stress', majority_vote, 10, [], ['sleep', 'living', 'basics', 'academic', 'study', 'career'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual       0    1    2    total
--------  ----  ---  ---  -------
0          373    0    0      373
1          358    0    0      358
2          369    0    0      369
total     1100    0    0     1100

Non-empty labels count: 3
Average accuracy: 0.5593939393939394
p_prediced is 0 for label 1 . Subtracted 1 from non_empty_label.
p_prediced is 0 for label 2 . Subtracted 1 from non_empty_label.
Precision: 0.11303030303030304
Recall: 1.0
Macro-averaging F-score: 0.20310372992104547


Naive Bayes result

In [34]:
# kNN
stress_copy = stress.copy()
result = naive_bayes_stratified(stress_copy, 10, 'stress', [], ['sleep', 'living', 'basics', 'academic', 'study', 'career'])
print(result)
print()
label_list = result.get_column_data('actual')
if 'total' in label_list:
    label_list.remove('total')

# Find non-empty labels
sum_of_actual_labels = result.get_column_data('total')
non_empty_label = 0
for i in sum_of_actual_labels:
    if i != 0:
        non_empty_label += 1

if result[result.row_count() - 1]['actual'] == 'total':
    non_empty_label -= 1

print("Non-empty labels count:", non_empty_label)

# Accuracy
total_accuracy = 0
for i in label_list: 
    num = accuracy(result, i)
    if num != -1:
        total_accuracy += num
acc = total_accuracy / non_empty_label
print("Average accuracy:", acc)

# Precision
total_precision = 0
denum = non_empty_label
for i in label_list:
    num = precision(result, i)
    if num != -1:
        total_precision += num
    else:
        print("p_prediced is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        non_empty_label -= 1
precision_m = total_precision / denum
print("Precision:", precision_m)

# Recall
total_recall = 0
denum = non_empty_label
for i in label_list: # labels are 1, 2, 3
    num = recall(result, i)
    if num != -1:
        total_recall += num
    else:
        print("p_actual is 0 for label", i, ". Subtracted 1 from non_empty_label.")
        denum -= 1
recall_m = total_recall / non_empty_label
print("Recall:", recall_m)

# F-score
f_score = 2 * precision_m * recall_m / (precision_m + recall_m)
print("Macro-averaging F-score:", f_score)

actual      0    1    2    total
--------  ---  ---  ---  -------
0         327   22   24      373
1          22  317   19      358
2          26   20  323      369
total     375  359  366     1100

Non-empty labels count: 3
Average accuracy: 0.9193939393939394
Precision: 0.8791740059160489
Recall: 0.8791630723132959
Macro-averaging F-score: 0.879168539080679


### 2.3 Observations

* The result returned is very interesting. If I get rid of the social and extra attributes, the result for kNN is pretty bad. This may suggest that social and extra are two attributes that are highly correlated.
* This further affirms the fact that kNN is very sensitive to data skew.
* Naive Bayes, on the other hand, is not affected by this change. It still produces a very high accuracy, precision, recall, and f-measure.

# 4. Issues, Challenges, and Observations

1. Issues and Challenges
* I don't have any issues or challenges with this assignment.

2. Obersevations
* Overall, as shown in both Section 2 and 3, the kNN algorithm only works well if the parameter is set up correctly. This turns out to be quite tricky in some cases, especially when we have to investigate what is the best setting for the algorithm to work properly - which can be expensive sometimes.
* Naive Bayes, on the other hand, is much more robust and works well in most cases. However, the down size is that it runs pretty slow(er) than kNN.
* In terms of accuracy, precision, recall, and f-measure, Naive Bayes is the clear winner. It produces a much higher result than kNN most of the time. The only time when kNN has a better result is suring Section 2, when the kNN parameter is set up differntly. However, the variation between two algorithm results is not that significant in that case.
* Comparing to the previous assignment when I choose the train and test set using holdout, I find that k-stratified cross validation is much better. It produces a more representative test set, which in turn produces a better and more reliable result.
* For kNN, the higher the k value, the generally better result we will yield. However, it is not always the case. Sometimes, the result will be worse if we increase the k value. This is because the algorithm is very sensitive to data skew. If the data is skewed, the algorithm will not work properly. Hence, if we can put it in a loop and stop until the marginal increase is less 0, we can stop there and get the best result.
